In [ ]:
"""
this program is based on this tutorial: https://www.youtube.com/watch?v=xfzGZB4HhEE
by freecodecamp. it executes batch calls to the IEX Cloud API to get data about the prices
of stocks on the S&P 500. it recommends trades to the user by writing to an Excel
sheet using the xlsxwriter, pandas, math, numpy, and requests libs.
i am rewriting it here for the purpose of closer code explanation, and to remove
the extra stuff included in the original file, code as well as supplementary
explanation.
"""

In [ ]:
#imports:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv') # this imports a list of stocks on the S&P 500 from this machine

In [ ]:
from secrets import IEX_CLOUD_API_TOKEN # this imports the API token from IEX Cloud

In [ ]:
# This creates a Pandas dataframe and puts our value metric, stock symbol columns into this data structure
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
#final_dataframe = pd.DataFrame(columns = my_columns) we don't need to initialize the dataframe yet
#final_dataframe

In [ ]:
# then we add the stock data (price, etc) to the dataframe via batch API calls
# first we define a special function:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
# then we add the stock symbols to the dataframe from the csv
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)): # creates a list of stocks in string format from the csv
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns = my_columns) # initializing the dataframe

for symbol_string in symbol_strings: # executes batch API calls to fetch the real-time stock prices from IEX cloud
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(# adds the 
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ],
        index = my_columns),
        ignore_index = True
    )
final_dataframe

In [ ]:
# now we ask user for how much money they want to invest
portfolio_size = input('Enter the value of your portfolio: ')
try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print('Please enter a numeric value.')
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)
# this is cool. never seen "try/except" before

In [ ]:
# now we generate the relative amounts of each stock to buy, based on the user's investment size, and add them to dataframe
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
#the .loc() method returns the rows/columns indexed by the second arg. this is a hard statement to read (visually), but it is quite simple mathematically

In [ ]:
# now we generate the excel file which contains our output (trading recs)
writer = pd.ExcelWriter('TradingRecs.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Trade Recommendations', index = False)
# formatting the spreadsheet:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border_color': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border_color': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border_color': 1
    }
)

column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Cap', dollar_format],
    'D': ['Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Trade Recommendations'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Trade Recommendations'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

# and saving it:
writer.save()